### data augumentation using Keras
on MNIST data


In [ ]:
import numpy as np
#from tensorflow import keras
import keras

print('keras version:', keras.__version__)
print('keras backend:', keras.backend.backend())
print('keras image format:', keras.backend.image_data_format())

In [ ]:
data = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = data.load_data()

In [ ]:
# show information about the downloaded data

print('train shape:', x_train.shape)
print('train y:', y_train.shape)
print('test_shape:', x_test.shape)
print('test y:', y_test.shape)


num_labels = len(np.unique(y_train))
image_size = x_train.shape[2]

print('num of labels:', num_labels)
print('image size:', image_size)

# calculate input shape and number of channels
is_channels_first = (keras.backend.image_data_format() == 'channels_first')
shape_len = len(x_train.shape)

if shape_len == 3:
    num_channels = 1
else:
    num_channels = 3

if is_channels_first:
    input_shape = (num_channels , image_szie , image_size)
else:
    input_shape = ( image_size , image_size , num_channels)

print('input shape:', input_shape)

In [ ]:
# convert the shape of data depending on the image data format

if is_channels_first :
    x_train2 = x_train.reshape(x_train.shape[0], num_channels, image_size, image_size)
    x_test2 = x_test.reshape(x_test.shape[0], num_channels, image_size, image_size)
else:
    x_train2 = x_train.reshape(x_train.shape[0], image_size, image_size, num_channels)
    x_test2 = x_test.reshape(x_test.shape[0], image_size, image_size, num_channels)


In [ ]:
# normalize the data: 0.0 to 1.0

x_train2 = x_train2.astype('float32') / 255
x_test2 = x_test2.astype('float32') / 255

#hot encode
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)

In [ ]:
# show info about reshaped data

print("revised x_train shape:", x_train2.shape)
print('revised y_train shape:', y_train2.shape)
print('revised x_test shape:', x_test2.shape)
print('revised y_test shape:', y_test2.shape)
print('input shape:',input_shape)

In [ ]:
num_hidden_layers = 512

print('hidden dense layer size:', num_hidden_layers)

In [ ]:
#model
model = keras.models.Sequential()
model.add( keras.layers.Conv2D(32, kernel_size=(3,3), input_shape=input_shape , activation='relu' ))
model.add( keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add( keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu' ))
model.add( keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add( keras.layers.Dropout(0.5))

model.add( keras.layers.Flatten())
model.add( keras.layers.Dense(num_hidden_layers, activation='relu'))

model.add( keras.layers.Dropout(0.5))
model.add( keras.layers.Dense(10, activation='softmax'))

# compile to model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show summary
model.summary()

In [ ]:
#train the model with train data for few runs to setup the weights

fit_history = model.fit(x_train2, y_train2,
epochs=10,
batch_size=128,
validation_data=(x_test2,y_test2)
)

In [ ]:
# create dirs to hold logs and models so you can review them later

import os
import time

# helper function to create unique sub folder
def create_folder(folder_name):
    if (not os.path.exists(folder_name)):
        os.makedirs(folder_name)
    new_dir = folder_name + "/{}".format(time.time())
    if (not os.path.exists(new_dir)):
        os.makedirs(new_dir)
    return new_dir

log_dir = create_folder('logs')
model_dir = create_folder('models')


In [ ]:
# define how to save snapshots of model states during training
checkpt_path=model_dir+'/va{val_acc:.5f}-ep{epoch:04d}-ac{acc:.5f}-vl{val_loss:.5f}-l{loss:.5f}.hdf5'
cp_callback = keras.callbacks.ModelCheckpoint(
  checkpt_path ,
  verbose=1
)

In [ ]:
# setup to capture log so that you can view it from tensorflow dashboard
# comment this out if you are NOT using tensorflow

#import tensorflow as tf
#tf_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


In [ ]:
# use datagen to 'augment' the training data
# tweak the parameters to get better results

datagen = keras.preprocessing.image.ImageDataGenerator(
  rotation_range = 25,
  width_shift_range=0.10,
  height_shift_range=0.10,
  zoom_range = 0.10
)

In [ ]:
batch_size = 128
epochs = 300

print('batch size:', batch_size)
print('epochs:', epochs)

In [ ]:
# run data augumentation based training
# add in tf_callback if using tensorflow so you can monitor the progress using tf dashboard

fit_history2 = model.fit_generator(
  datagen.flow(x_train2,y_train2,batch_size=batch_size),
  steps_per_epoch = int(len(x_train2) / batch_size),
  epochs = epochs,
  validation_data = (x_test2, y_test2),
  #workers=4, 
  callbacks=[cp_callback]
)